# DBT Cell Magic

> provides the `%%dbt` cell magic for creating models and analyses and
  `%dbtconfig` to configure the _dbt project dir_ and _dbt profiles dir_

In [ ]:
#| default_exp dbt_cellmagic

In [ ]:
#| include: false
#| notest
%reload_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| exporti

import IPython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic, register_line_magic

In [ ]:
#| exporti
from pathlib import Path
from typing import Union, List, Dict, Optional
import os
import shutil

In [ ]:
#| exporti
IN_NBDBT_TEST = os.environ.get("IN_NBDBT_TEST", "False").lower() == "true"
NBDBT_DEBUG = False

In [ ]:
#| exporti
from fastcore.all import patch

In [ ]:
#| exporti
# from dbt.main import parse_args, adapter_management
from dbt.task.compile import CompileTask
from dbt.contracts.results import RunExecutionResult
import dbt.flags
import dbt.tracking

In [ ]:
#| exporti
dbt.tracking.active_user = dbt.tracking.User(None)
dbt.flags.INDIRECT_SELECTION = "eager"
dbt.flags.WRITE_JSON = True
dbt.flags.QUIET = True  # silent

In [ ]:
#| exporti

# from fal import FalDbt
from nbdbt2.fal.dbt.integration.project import FalDbt
# import faldbt.lib as fallib
import nbdbt2.fal.dbt.integration.lib as fallib
import nbdbt2.fal.dbt.integration.project as falproject

In [ ]:
#| exporti

import pandas as pd

In [ ]:
#| export
nbdbt_config = {
    "profiles_dir": "~/.dbt"
    if os.environ.get("DBT_PROFILES_DIR") is None
    else os.environ.get("DBT_PROFILES_DIR"),
    "project_dir": None,
    "notebook": None,
    "limit_default": 1000,
}

In [ ]:
#| exporti
# Add cache
import json
import hashlib

CACHE_DIR = ".nbdbt"


def load_cache(cache_name):

    cache_path = Path(nbdbt_config["project_dir"]) / CACHE_DIR / cache_name
    if cache_path.exists():
        with open(cache_path, "r") as f:
            cache_contents = f.read()
            try:
                cache_info = json.loads(cache_contents)
            except json.JSONDecodeError as e:
                if NBDBT_DEBUG:
                    raise json.JSONDecodeError(e.msg, e.doc, e.pos)
                else:
                    return None
            else:
                return cache_info

    else:
        return None


def update_cache(dmo):
    if dmo._compiled_path is None:
        raise ValueError("Could not update cache as compiled path is not set")

    cache_name = dmo.file
    cache_path = Path(nbdbt_config["project_dir"]) / CACHE_DIR / cache_name
    if not cache_path.exists():
        cache_dir = cache_path.parent
        try:
            cache_dir.mkdir(parents=True, exist_ok=True)
        except FileExistsError:
            raise FileExistsError(
                f"cache parent path exists as a file (delete it first): {cache_dir}"
            )
    compiled_path = Path(dmo.project_dir) / dmo._compiled_path
    cache_info = {
        "formatted_sql_hash": hashlib.sha1(dmo._formatted_sql.encode()).hexdigest(),
        "compiled_path": dmo._compiled_path,
        "compiled_sql_mtime": compiled_path.stat().st_mtime,
    }
    with open(cache_path, "w") as f:
        cache_contents = json.dumps(cache_info)
        f.write(cache_contents)


def check_cache(dmo, cache):
    cache_formatted_sql_hash = cache["formatted_sql_hash"]
    formatted_sql_hash = hashlib.sha1(dmo._formatted_sql.encode()).hexdigest()
    if cache_formatted_sql_hash != formatted_sql_hash:
        return ["update_sql"]

    compiled_path = Path(dmo.project_dir) / cache["compiled_path"]
    if not compiled_path.exists():
        return ["update_model"]
    compiled_path_mtime = compiled_path.stat().st_mtime
    cache_compiled_path_mtime = cache["compiled_sql_mtime"]
    if compiled_path_mtime != cache_compiled_path_mtime:
        return ["update_model"]
    return []


def update_dmo(dmo, cache):
    dmo._compiled_path = cache["compiled_path"]

In [ ]:
#| export
def clear_cache(project_dir=None):
    """Removes all existing nbdbt cache entries"""
    project_dir = nbdbt_config["project_dir"] if project_dir is None else project_dir

    if project_dir is None:
        raise ValueError(
            "Please specify the dbt project dir as a parameter or as a %dbtconfig line magic"
        )
    cache_path = Path(project_dir) / CACHE_DIR
    if not cache_path.exists():
        return
    if cache_path.is_file():
        cache_path.unlink()
        return
    shutil.rmtree(cache_path, ignore_errors=True)

In [ ]:
#| export


@magic_arguments.magic_arguments()
@magic_arguments.argument(
    "-d",
    "--profile",
    type=str,
    default=None,
    help=("If provided, override the dbt profiles directory (default: '~/.dbt')"),
)
@magic_arguments.argument(
    "-p",
    "--project",
    type=str,
    default=None,
    help=("Set the dbt project directory"),
)
@magic_arguments.argument(
    "-n",
    "--notebook",
    type=str,
    default=None,
    help=("Set the notebook path"),
)
@magic_arguments.argument(
    "-l",
    "--limit",
    type=int,
    default=1000,
    help=("Set the default sql row limit"),
)
@register_line_magic("dbtconfig")
def config_dbt(line):
    if IN_NBDBT_TEST:
        return None
    try:
        from dbt.task.compile import CompileTask
    except ImportError:
        return "'dbt-core' not installed. Did you run 'pip install dbt-core'?"
    line_args = magic_arguments.parse_argstring(config_dbt, line)
    if line_args.profile is not None:
        nbdbt_config["profiles_dir"] = line_args.profile
    if line_args.project is not None:
        nbdbt_config["project_dir"] = line_args.project
    if line_args.notebook is not None:
        nbdbt_config["notebook"] = line_args.notebook

    nbdbt_config["limit"] = line_args.limit

In [ ]:
#| export


class DbtMagicObject:
    def __init__(
        self,
        raw_sql: str,  # sql string
        file: str,  # path to sql file (relative to dbt project dir)
        limit: int,  # limit row default
        project_dir: Optional[Union[str, Path]] = None,  # dbt project dir
        notebook_name: Optional[str] = None,  # name of notebook
        profile_dir: Optional[Union[str, Path]] = None,  # dbt profiles dir
    ):
        """Create a holder of dbt cell magic parameters"""
        self.raw_sql = raw_sql
        self.file = file
        self.limit = nbdbt_config["limit"] if limit == -1 else limit
        project_dir = (
            nbdbt_config["project_dir"] if project_dir is None else project_dir
        )
        if project_dir is None:
            raise ValueError("Missing value for  --project")
        self.project_dir = (
            Path(project_dir) if type(project_dir) == str else project_dir
        )

        notebook_name = (
            nbdbt_config["notebook"] if notebook_name is None else notebook_name
        )
        self.notebook_name = notebook_name

        profile_dir = (
            nbdbt_config["profiles_dir"] if profile_dir is None else profile_dir
        )
        self.profile_dir = (
            Path(profile_dir) if type(profile_dir) == str else profile_dir
        )

        # dynamic runtime attributes
        self._compiled_path: Optional[str] = None
        self._df_result: Optional[pd.DataFrame] = None
        

In [ ]:
#| export
@patch(as_prop=True)
def _formatted_sql(self: DbtMagicObject) -> str:
    """Add auto generated notice for nbdbt generated sql"""
    if not self.notebook_name:
        contents = "-- AUTOGENERATED! DO NOT EDIT!\n" + self.raw_sql
    else:
        contents = (
            f"-- AUTOGENERATED! DO NOT EDIT! File to edit: {self.notebook_name} (unless otherwise specified).\n"
            + self.raw_sql
        )
    return contents

In [ ]:
#| export
@patch(as_prop=True)
def _compiled_sql(self: DbtMagicObject) -> str:
    """Add auto generated notice for nbdbt generated sql"""
    if self._compiled_path is None:
        return None
    compiled_path = Path(self.project_dir) / self._compiled_path
    if not compiled_path.exists():
        return None
    with open(compiled_path, "r") as f:
        contents = f.read()
        return contents
    return None

In [ ]:
#| export
@patch
def _write_sql(self: DbtMagicObject) -> None:
    """Write sql to file"""
    path = self.project_dir / self.file
    parent = path.parent
    try:
        parent.mkdir(parents=True, exist_ok=True)
    except FileExistsError:
        raise FileExistsError(
            f"Sql file write failure: Existing file in parent path ${parent}"
        )

    with open(path, "w") as f:
        f.write(self._formatted_sql)

In [ ]:
#| export
@patch
def _compile_model(self: DbtMagicObject) -> None:
    """Compile model and store compile result"""
    faldbt = FalDbt(
        profiles_dir=str(self.profile_dir), 
        project_dir=str(self.project_dir),
        select=[self.file]
    )
    profile_target = faldbt._profile_target

    task = faldbt._compile_task
    compile_result = task.run()
    self._compiled_path = compile_result.results[0].node.compiled_path

In [ ]:
#| export
@patch
def _exec_faldbt_ref(self: DbtMagicObject, limit) -> None:
    """Execute sql and return df"""
    if self._compiled_sql is None:
        raise ValueError("Model compilation step has not been executed")

    faldbt = FalDbt(
        profiles_dir=str(self.profile_dir), project_dir=str(self.project_dir)
    )
    profile_target = faldbt._profile_target
    # adapter_response, result
    limit = self.limit if limit == -1 else limit
    if limit is None:
        exec_sql = self._compiled_sql
    else:
        ctx_name = "xxx_yyy_zzz"
        exec_sql = f"""with {ctx_name} as
        (
          {self._compiled_sql}
        )
        select * from {ctx_name} 
        limit {limit}
        """

    result = fallib.execute_sql(
        str(self.project_dir), str(self.profile_dir), profile_target, exec_sql
    )
    df_result = result
    self._df_result = df_result

In [ ]:
#| export
@patch
def ref(self: DbtMagicObject, limit=-1) -> pd.DataFrame:
    self._exec_faldbt_ref(limit)
    return self._df_result

In [ ]:
#| exporti
from nbdbt2.fal.dbt.integration.project import _DbtTestableNode

In [ ]:
#| export

@patch(as_prop=True)
def schema(self: _DbtTestableNode) -> pd.DataFrame:
    profiles_dir = nbdbt_config["profiles_dir"]
    project_dir = nbdbt_config["project_dir"]
    faldbt = FalDbt(profiles_dir=profiles_dir, project_dir=project_dir)
    profile_target = faldbt._profile_target

    node = self.node

    adapter = fallib._get_adapter(
        faldbt.project_dir, faldbt.profiles_dir, profile_target
    )

    # adapter.type() == 'bigquery'
    # if adapter.type() != "bigquery":
    #     raise NotImplementError("No support yet for any other adapter except BigQuery")
    #     return None

    # relation = fallib._get_target_relation(
    #     node,
    #     faldbt.project_dir,
    #     faldbt.profiles_dir,
    #     profile_target=faldbt._profile_target,
    # )
    relation = fallib._get_target_relation(adapter, node)

    info_schema = relation.information_schema()

    column_schema = info_schema.from_relation(relation, "COLUMNS")

    column_table = column_schema.render()

    table_name = relation.table

    fetch_schema_sql = f"""
    with schema_columns as
    ( select *
    from {column_table}
    where table_name = '{table_name}'
    )
    select * 
    from schema_columns
    """

    result = fallib.execute_sql(
        project_dir, profiles_dir, faldbt._profile_target, fetch_schema_sql
    )
    return result

In [ ]:
#| export
@magic_arguments.magic_arguments()
@magic_arguments.argument(
    "-a",
    "--assign",
    type=str,
    default=None,
    help=("If provided, save the output to this variable instead of displaying it."),
)
@magic_arguments.argument(
    "-p",
    "--project",
    type=str,
    default=None,
    help=("dbt project directory"),
)
@magic_arguments.argument(
    "-n",
    "--notebook",
    type=str,
    default=None,
    help=("notebook source file"),
)
@magic_arguments.argument(
    "-l",
    "--limit",
    type=int,
    default=-1,
    help=("sql limit default"),
)
@magic_arguments.argument(
    "file",
    type=str,
    help=("file path to write to"),
)
@register_cell_magic("dbt")
def write_dbt(line, cell):
    if IN_NBDBT_TEST:
        return None
    try:
        from dbt.task.compile import CompileTask
    except ImportError:
        return "'dbt-core' not installed. Did you run 'pip install dbt-core'?"
    line_args = magic_arguments.parse_argstring(write_dbt, line)
    dmo = DbtMagicObject(
        cell, line_args.file, line_args.limit, line_args.project, line_args.notebook
    )
    cache = load_cache(line_args.file)
    if cache is None:
        dmo._write_sql()
        dmo._compile_model()
        update_cache(dmo)
    else:
        check_results = check_cache(dmo, cache)
        if "update_sql" in check_results:
            dmo._write_sql()
            dmo._compile_model()
        elif "update_model" in check_results:
            dmo._compile_model()
        if len(check_results) > 0:
            update_cache(dmo)
        else:
            update_dmo(dmo, cache)

    if line_args.assign:
        IPython.get_ipython().push({line_args.assign: dmo})
        return None
    results = dmo.ref()
    return results

In [ ]:
#| include: false
test_project = "test_dbt"
test_project_dir = f"../{test_project}"
test_model1 = "analyses/sample1.sql"
test_model2 = "analyses/sample2.sql"

In [ ]:
#| include: false
!rm -f {test_project_dir}/{test_model1}
!rm -f {test_project_dir}/target/compiled/{test_project}/{test_model1}
!rm -rf {test_project_dir}/{CACHE_DIR}

In [ ]:
%dbtconfig -d ../test_profiles -p ../test_dbt -n nbs/00_dbt_cellmagic.ipynb

In [ ]:
%%dbt analyses/sample1.sql

select *
from {{ ref('my_first_dbt_model') }}
where id is not null

13:10:01 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:01  Registered adapter: duckdb=1.7.0
13:10:01  Running with dbt=1.7.4
13:10:01  Registered adapter: duckdb=1.7.0
13:10:01  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:02  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:02  The selection criterion 'analyses/sample1.sql' does not match any nodes
13:10:02 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'
13:10:02  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:02  
13:10:02  Concurrency: 1 threads (target='dev')
13:10:02  
13:10:02  Compiled node 'sample1' is:
-- AUTOGENERATED! DO NOT EDIT! File to edit: nbs/00_dbt_c

,id
0,1


In [ ]:
#| include: false
# skip_test
assert _.equals(pd.DataFrame(data={"id": [1]}))

In [ ]:
#| include: false
# skip_test
# check that analyses file was created
assert (Path(test_project_dir) / test_model1).exists()

In [ ]:
#| include: false
# skip_test
# check that compiled file was created
assert (
    Path(test_project_dir) / "target" / "compiled" / test_project / test_model1
).exists()

In [ ]:
#| include: false
# skip_test
# check that cache file was created
assert (Path(test_project_dir) / CACHE_DIR / test_model1).exists()

In [ ]:
#| include: false
!rm -f {test_project_dir}/{test_model2}
!rm -f {test_project_dir}/target/compiled/{test_project}/{test_model2}

In [ ]:
if "sample2" in globals():
    del sample2

In [ ]:
%%dbt -a sample2 analyses/sample2.sql 

select *
from {{ ref('my_second_dbt_model') }}
where id is not null

13:10:04 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:04  Registered adapter: duckdb=1.7.0
13:10:04  Running with dbt=1.7.4
13:10:04  Registered adapter: duckdb=1.7.0
13:10:04  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:05  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:05 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'
13:10:05  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:05  
13:10:05  Concurrency: 1 threads (target='dev')
13:10:05  
13:10:05  Compiled node 'sample2' is:
-- AUTOGENERATED! DO NOT EDIT! File to edit: nbs/00_dbt_cellmagic.ipynb (unless otherwise specified).

select *
from "dev"."main"."my_secon

In [ ]:
#| include: false
# skip_test
# check that analyses file was created
assert (Path(test_project_dir) / test_model2).exists()

In [ ]:
#| include: false
# check that compiled file was created
(Path(test_project_dir) / "target" / "compiled" / test_project / test_model2).exists()

True

In [ ]:
#| include: false
# skip_test
# check that compiled file was created
assert (Path(test_project_dir) / CACHE_DIR / test_model2).exists()

In [ ]:
#| include: false
# skip_test
assert "sample2" in globals()

In [ ]:
# skip_test
results = sample2.ref()

13:10:05 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:05  Registered adapter: duckdb=1.7.0
13:10:05  Running with dbt=1.7.4
13:10:05  Registered adapter: duckdb=1.7.0
13:10:05  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:07  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:07 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'


In [ ]:
# skip_test
results

,id
0,1


In [ ]:
#| include: false
# skip_test
assert results.equals(pd.DataFrame(data={"id": [1]}))

In [ ]:
#| include: false
# skip_test
cache_dir = Path(nbdbt_config["project_dir"]) / CACHE_DIR

In [ ]:
#| include: false
# skip_test
assert cache_dir.exists()

In [ ]:
%%dbt -a my_third_model -n notebooks/index.ipynb models/my_third_model.sql
select *
from {{ ref('my_second_dbt_model') }}

13:10:07 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:07  Registered adapter: duckdb=1.7.0
13:10:07  Running with dbt=1.7.4
13:10:07  Registered adapter: duckdb=1.7.0
13:10:07  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:08  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:08 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'
13:10:08  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:08  
13:10:08  Concurrency: 1 threads (target='dev')
13:10:08  
13:10:08  Compiled node 'my_third_model' is:
-- AUTOGENERATED! DO NOT EDIT! File to edit: notebooks/index.ipynb (unless otherwise specified).
select *
from "dev"."main"."my_seco

In [ ]:
# skip_test
my_third_model

<__main__.DbtMagicObject>

In [ ]:
#| include: false
# skip_test
from pathlib import Path

assert Path("../test_dbt/models/my_third_model.sql").exists()
assert Path(
    "../test_dbt/target/compiled/test_dbt/models/my_third_model.sql"
).exists()

The `ref` method on `DbtMagicObject`  allows us to run the query and save the results into a dataframe.

In [ ]:
# skip_test
results = my_third_model.ref()
results  # dataframe

13:10:08 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:08  Registered adapter: duckdb=1.7.0
13:10:08  Running with dbt=1.7.4
13:10:08  Registered adapter: duckdb=1.7.0
13:10:08  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:10  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:10 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'


,id
0,1


In [ ]:
# skip_test
faldbt = FalDbt(nbdbt_config["project_dir"], nbdbt_config["profiles_dir"])
model0 = faldbt.list_models()[0]
source0 = faldbt.list_sources()[0]

13:10:10 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:10  Registered adapter: duckdb=1.7.0
13:10:10  Running with dbt=1.7.4
13:10:10  Registered adapter: duckdb=1.7.0
13:10:10  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:11  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:11 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'


In [ ]:
# skip_test
model0.schema

13:10:11 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:11  Registered adapter: duckdb=1.7.0
13:10:11  Running with dbt=1.7.4
13:10:11  Registered adapter: duckdb=1.7.0
13:10:11  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:12  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:12 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'
13:10:12  Unable to do partial parsing because config vars, config profile, or config target have changed


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,dev,main,my_third_model,id,1,None,YES,INTEGER,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# skip_test
source0.schema

13:10:14 [fal]: [WARNING]: Could not read dbt run_results artifact
13:10:14  Registered adapter: duckdb=1.7.0
13:10:14  Running with dbt=1.7.4
13:10:14  Registered adapter: duckdb=1.7.0
13:10:14  Performance info: /home/butch2/play/experiments/nbtools/nbdbt2/test_dbt/target/perf_info.json
13:10:15  Found 4 models, 4 analyses, 4 tests, 1 source, 0 exposures, 0 metrics, 391 macros, 0 groups, 0 semantic models
13:10:15 [fal]: [WARNING]: Could not read dbt sources artifact: Runtime Error
  Could not read FreshnessExecutionResultArtifact at "target/sources.json" as JSON: [Errno 2] No such file or directory: 'target/sources.json'


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,dev,main,my_sample_source,id,1,None,YES,INTEGER,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
#| include: false
# skip_test
clear_cache()

In [ ]:
#| include: false
# skip_test
assert not cache_dir.exists()